In [1]:
#======================================================================================================
# *WandB 툴을 이용하여, HyperPrameter sweeps 하는 예제임(http://wandb.ai 에 회원가입 해야 함)
# - bert BertForSequenceClassification finetuning 예제를 이용함
#  
# == HyperPrameter sweeps 과정 ==
#
# 1. 로그인
# import wandb
# wandb.login()  # 로그인 => wandb.ai 사이트에 로그인 후, API Key 입력해야 함
#
# 2. sweep_config 설정
# sweep_config ={ 
#       'method': 'random', #grid, random
#       'metric':{
#           'name': 'val_accuracy', #loss
#           'goal': 'maximize'      # loss일때는 minimize로 설정해야함
#       },
#      'parameters':{
#           'learning_rate':{
#               'distribution': 'uniform',
#               'min' : 0,
#               'max': 7e-5
#           },
#           'batch_size':{
#               'values' : [8, 16, 32]
#           },
#           'epochs':{
#               'values' : [2, 3, 4]
#           }
#       }
#}
#
#sweep_id = wandb.sweep(sweep_config, project="bert-wb-test")
# 3. wandb 초기화 
# - wandb.init(config=config)
#
# 4. wandb 로그설정
# - wandb.log({"val_accuracy": total_test_correct / total_test_len})
#
# 5. wand 실행
# - wandb.agent(sweep_id, train, count=3)
#
# 6. wand 종료
# -wandb.finish()
#======================================================================================================
#!pip install wandb -qqq
import wandb
wandb.login()  # 로그인 => wandb.ai 사이트에 로그인 후, API Key 입력해야 함

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kobongsoo (use `wandb login --relogin` to force relogin)


True

In [2]:
#======================================================================================================
# bert BertForSequenceClassification finetuning 정의
# 참고 사이트 : https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/
#
#======================================================================================================

import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, BertModel, AdamW, get_linear_schedule_with_warmup
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch.nn.functional as F

import sys
sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging
from tqdm.notebook import tqdm
import os

logger = mlogging(loggername="bertfttrain", logfilename="../../log/bertftmultitrain")
device = GPU_info()
seed_everything(111)



logfilepath:bwdataset_2022-04-14.log
logfilepath:qnadataset_2022-04-14.log
logfilepath:../../log/bertftmultitrain_2022-04-14.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [ ]:
##############################################################################################
# 변수들 설정
# - model_path : from_pretrained() 로 호출하는 경우에는 모델파일이 있는 폴더 경로나 
#          huggingface에 등록된 모델명(예:'bert-base-multilingual-cased')
#          torch.load(model)로 로딩하는 경우에는 모델 파일 풀 경로
#
# - vocab_path : from_pretrained() 호출하는 경우에는 모델파일이 있는 폴더 경로나
#          huggingface에 등록된 모델명(예:'bert-base-multilingual-cased')   
#          BertTokenizer() 로 호출하는 경우에는 vocab.txt 파일 풀 경로,
#
# - OUTPATH : 출력 모델, vocab 저장할 폴더 경로
#############################################################################################

#model_path = 'model/kobertmodel/'
#vocab_path = 'model/kobertmodel/vocab/vocab.txt'
#OUTPATH = 'model/classification/kobert-ft-es-cfmodel/'


vocab_path = '../../model/bert/bmc-fpt-wiki_20190620_mecab_false_0311-nouns-0327/'

# tokeniaer 및 model 설정
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# strip_accents=False : True로 하면, 가자 => ㄱ ㅏ ㅈ ㅏ 식으로 토큰화 되어 버림(*따라서 한국어에서는 반드시 False)
# do_lower_case=False : # 소문자 입력 사용 안함(한국어에서는 반드시 False)
#tokenizer = BertTokenizer(vocab_file=vocab_path, strip_accents=False, do_lower_case=False) 
tokenizer = BertTokenizer.from_pretrained(vocab_path, do_lower_case=False)



In [ ]:

model_path = '../../model/bert/bmc-fpt-wiki_20190620_mecab_false_0311-nouns-0327/'
OUTPATH = '../../model/classification/bmc-fpt-wiki_20190620_mecab_false_0311-nouns-0327-nscm-0414-1/'

def load_model():
    # 레벨을 1개만 선택하는 경우
    model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)
    #model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=6)

    # 레벨을 멀티로 선택해야 하는 경우
    #model = BertForSequenceClassification.from_pretrained(model_path, problem_type="multi_label_classification",num_labels=6)

    #기존 모델 파일을 로딩하는 경우    
    #model = torch.load(model_path) 

    model.to(device)
    
    return model

In [5]:
# tokenier 테스트
print(len(tokenizer))
print(tokenizer.encode("눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다"))
print(tokenizer.convert_ids_to_tokens(1000))
print(tokenizer.convert_tokens_to_ids('날씨'))

167550
[101, 9034, 10530, 124997, 11018, 125215, 10739, 69708, 42428, 10459, 10020, 129937, 10892, 132489, 12508, 49137, 102]
৭
123665


In [6]:
'''
# loss 만 출력할때.
# - wandb 관련 hyper parameter들 설정
wandb.init(
        project='bert-ft-multiclassification',
        config={
            "epochs": 3,
            "batch_size": 32,
            "learning_rate": 2e-5,
        })

config = wandb.config
'''

#=======================================================================
# sweep_config 설정
# - 
# - Hyper parameter 설정값 찾을때 설정함
# - 일반적으로 val_accuracy 최대가 될때 hyper-parameter들을 찾도록 metric를 설정함
#   (*이때 val_accuracy 명칭은 뒤에 wandb.log 함수할때와 같은 명칭을 사용해야 함)
sweep_config ={ 
        'method': 'random', #grid, random
        'metric':{
            'name': 'val_accuracy', #loss
            'goal': 'maximize'      # loss일때는 minimize로 설정해야함
        },
        'parameters':{
            'learning_rate':{
                'distribution': 'uniform',
                'min' : 0,
                'max': 2e-4
            },
            'batch_size':{
                'values' : [8, 16, 32]
            },
            'epochs':{
                'values' : [2, 3, 4]
            }
        }
}


sweep_id = wandb.sweep(sweep_config, project="bert-wb-test")
#=======================================================================

Create sweep with ID: 0g05iq6k
Sweep URL: https://wandb.ai/kobongsoo/bert-wb-test/sweeps/0g05iq6k


In [7]:
# 학습 data loader 생성
def build_dataset(batch_size):
    from os import sys
    sys.path.append('..')
    from myutils import ClassificationCSVCorpus, ClassificationDataset, data_collator
    from torch.utils.data import DataLoader, RandomSampler

    #############################################################################
    # 변수 설정
    #############################################################################
    max_seq_len = 128   # 글자 최대 토큰 길이 해당 토큰 길이 이상은 잘린다.
    #batch_size = 32       # 배치 사이즈(64면 GUP Memory 오류 나므로, 32 이하로 설정할것=>max_seq_length 를 줄이면, 64도 가능함)

    # 훈련할 csv 파일
    #file_fpath = 'korpora/감성대화말뭉치/감성대화말뭉치(최종데이터)_renew_labelenc_Training.csv'
    file_fpath = '../../korpora/nsmc/ratings_train.txt'
    column_num = 3           # .csv 파일에 컬럼수(예: text, label만 있으면 =2)
    csvfile = 0              # 0:tsv 파일, 1: csv 파일
    label_list = ["0", "1"]  # .csv 파일에 레벨 목록( list로 입력해야 함)
    #label_list = ["0", "1", "2", "3", "4", "5"]  # .csv 파일에 레벨 목록( list로 입력해야 함)
    cache = False   # 캐쉬파일 생성할거면 True로 (True이면 loding할때 캐쉬파일있어도 이용안함)
    #############################################################################

    # corpus 파일 설정
    corpus = ClassificationCSVCorpus(column_num=column_num, iscsvfile=csvfile, label_list=label_list)

    # 학습 dataset 생성
    dataset = ClassificationDataset(file_fpath=file_fpath, max_seq_length=max_seq_len, tokenizer=tokenizer, corpus=corpus, overwrite_cache=cache)


    # 학습 dataloader 생성
    train_loader = DataLoader(dataset, 
                              batch_size=batch_size, 
                              #shuffle=True, # dataset을 섞음
                              sampler=RandomSampler(dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                              collate_fn=data_collator, # dataset을 tensor로 변환(예시 {'input_ids':tensor[0,1,2,3,1,], 'token_type_id:tensor[0,0,0,0,0], 'attention_mask:tensor[1,1,1,1,1], 'labels':tensor[5]}
                              num_workers=4)

    # 평가 dataset 생성
    #file_fpath = 'korpora/감성대화말뭉치/감성대화말뭉치(최종데이터)_renew_labelenc_Validation.csv'
    file_fpath = '../../korpora/nsmc/ratings_test.txt'
    dataset = ClassificationDataset(file_fpath=file_fpath, max_seq_length=max_seq_len, tokenizer=tokenizer, corpus=corpus, overwrite_cache=cache)


    # 평가 dataloader 생성
    eval_loader = DataLoader(dataset, 
                              batch_size=batch_size, 
                              #shuffle=True, # dataset을 섞음
                              sampler=RandomSampler(dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                              collate_fn=data_collator, # dataset을 tensor로 변환(예시 {'input_ids':tensor[0,1,2,3,1,], 'token_type_id:tensor[0,0,0,0,0], 'attention_mask:tensor[1,1,1,1,1], 'labels':tensor[5]}
                              num_workers=4)
    
    return train_loader, eval_loader

In [8]:
# 학습 시작
def Train_epoch(config, 
                model,
                train_loader,
                eval_loader,
               ):   
    ##################################################
    # 변수 설정
    ##################################################
    epochs = config.epochs            # epochs
    learning_rate = config.learning_rate  # 학습률
    ##################################################

    # optimizer 적용
    optimizer = AdamW(model.parameters(), 
                     lr=learning_rate, 
                     eps=1e-8) # 0으로 나누는 것을 방지하기 위한 epsilon 값(10^-6 ~ 10^-8 사이 이값 입력합)

    # 총 훈련과정에서 반복할 스탭
    total_steps = len(train_loader)*epochs
    
    num_warmup_steps = int(total_steps*0.1)
    p_itr = int(len(train_loader)*0.1)          
        
    logger.info('total_steps: {}, warnup_steps: {}, p_itr: {}'.format(total_steps, num_warmup_steps, p_itr))

    # 스캐줄러 생성
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps=num_warmup_steps, 
                                                num_training_steps=total_steps)

    itr = 1
    total_loss = 0
    total_len = 0
    total_correct = 0
    list_training_loss = []
    list_acc_loss = []
    list_validation_acc_loss = []

    model.zero_grad()# 그래디언트 초기화
    for epoch in tqdm(range(epochs)):

        #model.train() # 훈련모드로 변환
        for data in tqdm(train_loader):
            model.train() # 훈련모드로 변환
            #optimizer.zero_grad()
            model.zero_grad()# 그래디언트 초기화

            # 입력 값 설정
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            token_type_ids = data['token_type_ids'].to(device)       
            labels = data['labels'].to(device)
            #print('Labels:{}'.format(labels))

            # 모델 실행
            outputs = model(input_ids=input_ids, 
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            labels=labels)

            # 출력값 loss,logits를 outputs에서 얻어옴
            loss = outputs.loss
            logits = outputs.logits
            #print('Loss:{}, logits:{}'.format(loss, logits))

            # optimizer 과 scheduler 업데이트 시킴
            loss.backward()   # backward 구함
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   # 그래디언트 클리핑 (gradient vanishing이나 gradient exploding 방지하기 위한 기법)
            optimizer.step()  # 가중치 파라미터 업데이트(optimizer 이동)
            scheduler.step()  # 학습률 감소

            # 정확도와 손실률 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
            # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
            # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
            with torch.no_grad():
                # 정확도와 총 손실률 계산
                pred = torch.argmax(F.softmax(logits), dim=1)
                correct = pred.eq(labels)
                total_correct += correct.sum().item()
                total_len += len(labels)    
                total_loss += loss.item()

                # 훈련 손실류 표기
                if itr % p_itr == 0:

                    logger.info('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Train Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))

                    # wandb 로그 기록
                    wandb.log({"Loss": total_loss/p_itr,
                              "train_accuracy": total_correct/total_len
                              })

                    list_training_loss.append(total_loss/p_itr)
                    list_acc_loss.append(total_correct/total_len)

                    total_loss = 0
                    total_len = 0
                    total_correct = 0

            
                    ####################################################################
                    # 주기마다 test(validataion) 데이터로 평가하여 손실류 계산함.
                    # 평가 시작
                    model.eval()

                    total_test_correct = 0
                    total_test_len = 0

                    #for data in tqdm(eval_loader):
                    for data in eval_loader:
                        # 입력 값 설정
                        input_ids = data['input_ids'].to(device)
                        attention_mask = data['attention_mask'].to(device)
                        token_type_ids = data['token_type_ids'].to(device)       
                        labels = data['labels'].to(device)

                        # 손실률 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
                        # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
                        # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
                        with torch.no_grad():
                            # 모델 실행
                            outputs = model(input_ids=input_ids, 
                                            attention_mask=attention_mask,
                                            token_type_ids=token_type_ids,
                                            labels=labels)

                            # 출력값 loss,logits를 outputs에서 얻어옴
                            #loss = outputs.loss
                            logits = outputs.logits

                            # 총 손실류 구함
                            pred = torch.argmax(F.softmax(logits), dim=1)
                            correct = pred.eq(labels)
                            total_test_correct += correct.sum().item()
                            total_test_len += len(labels)

                    list_validation_acc_loss.append(total_test_correct/total_test_len)
                    logger.info("[Epoch {}/{}] Validatation Accuracy:{}".format(epoch+1, epochs, total_test_correct / total_test_len))

                    # wandb 로그 기록
                    wandb.log({"val_accuracy": total_test_correct / total_test_len
                              })

                    ####################################################################
            
            itr+=1

        

In [9]:
def train(config=None):
    # wandb 초기화
    with wandb.init(config=config):
        config = wandb.config
        
        # 데이터 로더 생성
        train_loader, eval_loader = build_dataset(config.batch_size)
        
        # 모델 로딩
        tmodel = load_model()
        
        # 훈련 시작 
        Train_epoch(config, tmodel, train_loader, eval_loader)

In [10]:
# 훈련을 시작 (총 3번(epoch이 아님))
wandb.agent(sweep_id, train, count=3)

2022-04-14 13:13:25,079 - bertfttrain - INFO - [Epoch 3/3] Iteration 12168 -> Train Loss: 0.0845, Train Accuracy: 0.972


  0%|          | 0/1563 [00:00<?, ?it/s]

2022-04-14 13:14:19,908 - bertfttrain - INFO - [Epoch 3/3] Validatation Accuracy:0.87586
2022-04-14 13:15:25,608 - bertfttrain - INFO - [Epoch 3/3] Iteration 12636 -> Train Loss: 0.0882, Train Accuracy: 0.971


  0%|          | 0/1563 [00:00<?, ?it/s]

2022-04-14 13:16:20,665 - bertfttrain - INFO - [Epoch 3/3] Validatation Accuracy:0.87486
2022-04-14 13:17:28,113 - bertfttrain - INFO - [Epoch 3/3] Iteration 13104 -> Train Loss: 0.0826, Train Accuracy: 0.972


...

  0%|          | 0/1563 [00:00<?, ?it/s]

2022-04-14 16:31:54,377 - bertfttrain - INFO - [Epoch 4/4] Validatation Accuracy:0.49654
2022-04-14 16:33:04,896 - bertfttrain - INFO - [Epoch 4/4] Iteration 18720 -> Train Loss: 0.6932, Train Accuracy: 0.499


  0%|          | 0/1563 [00:00<?, ?it/s]

2022-04-14 16:33:59,495 - bertfttrain - INFO - [Epoch 4/4] Validatation Accuracy:0.50346


Loss,▆▂▅▅▅▅▄▆▇▄█▅▅▄▄▄▂▄▃▅█▃▂▂▂▂▂▂▁▁▂▁▂▁▁▁▁▁▂▁
train_accuracy,▅▆▃▅▇▅█▅▄▄▆▄▄▃▄▅▆▅▁▄▄▅▆▄▂▆▁▆▅▅▆█▅▄▄▆▆▇▂▄
val_accuracy,██▁▁█▁▁▁█▁█▁█▁███▁█▁▁█▁██▁██▁▁▁██▁▁▁▁▁▁█
Loss,0.69316
train_accuracy,0.49907
val_accuracy,0.50346


In [11]:
# wnandb 종료
wandb.finish()

In [ ]:
# 그래프로 loss 표기
#!pip install matplotlib
import matplotlib.pyplot as plt

plt.plot(list_training_loss, label='Train Loss')
plt.plot(list_acc_loss, label='Train Accuracy')
plt.legend()
plt.show()

In [ ]:
# train loss와 Validatiaon acc 출력
plt.plot(list_training_loss, label='Train Loss')
plt.plot(list_validation_acc_loss, label='Validatiaon Accuracy')
plt.legend()
plt.show()

In [ ]:
### 전체모델 저장
os.makedirs(OUTPATH)
#torch.save(model, OUTPATH + 'pytorch_model.bin') 
model.save_pretrained(OUTPATH)  # save_pretrained 로 저장하면 config.json, pytorch_model.bin 2개의 파일이 생성됨

# tokeinizer 파일 저장(vocab)
VOCAB_PATH = OUTPATH + 'vocab'
os.makedirs(VOCAB_PATH)
tokenizer.save_pretrained(VOCAB_PATH)

In [ ]:
# 최종 평가 시작
model.eval()

total_loss = 0
total_len = 0
total_correct = 0

for data in tqdm(eval_loader):
   
    # 입력 값 설정
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)       
    labels = data['labels'].to(device)
 
    # 모델 실행
    outputs = model(input_ids=input_ids, 
                   attention_mask=attention_mask,
                   token_type_ids=token_type_ids,
                   labels=labels)
    
     # 출력값 loss,logits를 outputs에서 얻어옴
    loss = outputs.loss
    logits = outputs.logits
    
    # 총 손실류 구함
    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test-accuracy: ', total_correct / total_len)
logger.info(f"Test-accuracy: {total_correct / total_len}")
    


In [ ]:
# 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능
'''
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, OUTPATH + 'all.tar')  
'''